In [1]:
import sys, time, re
import os
import pandas as pd
import pylab as plt
from importlib import reload
%matplotlib inline
import itertools as it
import numpy as np
from multiprocessing import Pool as ThreadPool
import  Levenshtein
from tqdm import tqdm_notebook as tqdm
import pickle
import gc
import sys
sys.path.append('../scripts')

import ml_utils as utils
reload(utils)

<module 'ml_utils' from '../scripts/ml_utils.py'>

### Spark test

In [2]:
APP_NAME='UIL'
sc = None
os.environ["PYSPARK_DRIVER_PYTHON"] = '/home/ildar/anaconda3/envs/ds/bin/python'
os.environ["PYSPARK_PYTHON"] = '/home/ildar/anaconda3/envs/ds/bin/python'

In [4]:
def prepareCtx():
    global sc
    if sc:
        sc.stop()
    
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setAppName(APP_NAME).setMaster('local[*]') #.setMaster('spark://bmi:7077')
            .set('spark.executor.memory', '2G')
            .set('spark.driver.memory', '12G')
#             .set('spark.driver.maxResultSize', '12G')
           )
    sc = SparkContext.getOrCreate(conf=conf)
        

prepareCtx()

In [ ]:
import random
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1
NUM_SAMPLES=100000

count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

In [13]:
lg, rg = G1, G2

In [16]:
%%time
def map_spread_mark(seed):
    lnode, rnode = seed[:2]

    seeds_collect = {}
    for l_neighbor in lg.neighbors(lnode):
        for r_neighbor in rg.neighbors(rnode):
            candidate = (l_neighbor, r_neighbor)
            seeds_collect[candidate] = seeds_collect.get(candidate, 0) + 1
    return seeds_collect

seeds = matches
res2 = list(map(map_spread_mark, seeds))
del res2

CPU times: user 6.52 s, sys: 348 ms, total: 6.87 s
Wall time: 6.87 s


In [20]:
%%time
class A:
    def __init__(self):
        self.lg = G1
        self.rg = G2
        self.seeds = matches
        self.seeds_collect={}

    def to_str(self, ln ,rn): return '%d|%d' % (ln, rn)

    def map_spread_mark(self, seeds):
        lnode, rnode = seed[:2]
        
        for l_neighbor in self.lg.neighbors(lnode):
            for r_neighbor in self.rg.neighbors(rnode):
                candidate = (l_neighbor, r_neighbor)
                self.seeds_collect[candidate] =self.seeds_collect.get(candidate, 0) + 1
#         return seeds_collect

    def start_parallel(self):
       
        distSeeds = sc.parallelize(self.seeds)
        res =  distSeeds.map(map_spread_mark).collect()
        return res
        
a = A()

res = a.start_parallel()
del res

CPU times: user 12.6 s, sys: 728 ms, total: 13.3 s
Wall time: 19.1 s


In [21]:
a.seeds_collect

{}

In [ ]:
def __spread_mark(lnode, rnode):
    # add one mark to all neighboring pairs of [i,j]
    if not seeds_collect:
        seeds_collect = {}
        old_marks = {}
        is_from_spread_marks=False
    else:
        is_from_spread_marks=True
    for l_neighbor in self.lg.neighbors(lnode):
        for r_neighbor in self.rg.neighbors(rnode):
            ID_str = self.__decide_seed(l_neighbor, r_neighbor)
            if not ID_str:
                continue

            val = self.score_map.get(ID_str)
            if not val:
         distSeeds       self.score_map[ID_str] = 1
                continue
            if ID_str not in old_marks:
                old_msarks[ID_str] = val
            self.score_map[ID_str] += 1
            seeds_collect[(l_neighbor, r_neighbor)] = val+1
    if is_from_spread_marks:
        return
    for seed, marks_count in seeds_collect.items():
        ID_str = self.to_str(*seed)
        m = (seed[0], seed[1], old_marks[ID_str])
        self.inactive_pairs.discard(m)
        self.inactive_pairs.add((seed[0], seed[1], marks_count))

def __spread_marks():
    # for all pairs[i, j] of A do
    print('start __spread_marks')
    seeds_collect = {}
    old_marks = {}
    for seed in tqdm(self.seeds):
        self.used.add(self.to_str(*seed))
        self.__spread_mark(*seed, seeds_collect = seeds_collect, old_marks = old_marks)
s
    for seed, marks_count in tqdm(seeds_collect.items()):
        ID_str = self.to_str(*seed)
        m = (seed[0], seed[1], old_marks[ID_str])
        self.inactive_pairs.discard(m)
        self.inactive_pairs.add((seed[0], seed[1], marks_count))

    # A <- None
    self.seeds.clear()
    print("Seed are expanded")

### Read graphs

In [6]:
%%time
reload(utils)
g1_fname = 'vk_lid_rid.csv'
g2_fname = 'inst_lid_rid.csv'
G1, G2 = utils.read_gs(g1_fname, g2_fname, from_raw = False)

G1 has all fname True
G2 has all fname False
CPU times: user 376 ms, sys: 40 ms, total: 416 ms
Wall time: 414 ms


In [ ]:
threshold=70
folder_type = 0
matches_file_name = 'matches_s_10_th_070_t_10-17_17:04:48.pickle'

In [7]:
threshold=91
folder_type = 0
matches_file_name = 'matches_s_03_th_091_t_10-19_23:31:15.pickle'

In [ ]:
threshold=81
folder_type = 0
matches_file_name = 'matches_s_03_th_081_t_10-20_00:10:59.pickle'

In [8]:
reload(utils)

matches, seeds_features = utils.read_matches(matches_file_name, threshold, folder_type, with_train=True)
print(utils.precision_recall(matches))

fets_total = len(set(seeds_features))
fs = len(set(seeds_features) - set(matches))
fets_total - fs, fs, 'total', fets_total

me/ildar/projects/pycharm/social_network_revealing/graphmatching/matches/no_repeat/091/matches_s_03_th_091_t_10-19_23:31:15.pickle
matches len 4124
(78897392, 608311198)
True 3980 False 144
(0.965082444228904, 0.16140151668761912)


(4121, 1113, 'total', 5234)

- train_data_matches_f86_1hop_ratio
- train_data_matches_f85_1hop
- train_data_matches_f86_1-2hop
- train_data_matches_f85_th91
- train_data_matches_f85_th81

In [ ]:
reload(utils)
utils.gen_train_data(matches, G1, G2, save_to = 'train_data_matches_f85_th81', threads = 3)

In [ ]:
features, labels = utils.load_train_data('train_data_matches_f85_th81') #train_data_matches_f86_1-2hop
print(len(features[0]))
len(features), len(labels)

### Generate true mapping

In [ ]:
utils.generate_true_mapping(from_raw=False)

In [ ]:
df = utils.read_combine_df(from_raw=False, merge_how='outer')

#### Features cache

In [ ]:
utils.f_set1s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G1.pickle'), "rb")))
utils.f_set2s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G2.pickle'), "rb")))

### ML models

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [ ]:
features_use = features
labels_use = labels

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(
#      features_use, labels_use, test_size=0.3, random_state=42)
x_train, y_train = features, labels

len(x_train[0]), len(features_use[0])

In [ ]:
forest = RandomForestClassifier(n_estimators = 400, criterion ='gini',
                        bootstrap = True, max_features = 'auto' ,n_jobs=4)
forest.fit(x_train, y_train)

In [ ]:
forest.score(x_test, y_test)

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(x_train, y_train)

In [ ]:
gbc.score(x_test, y_test)

In [ ]:
y_score = forest.predict(x_test)
y_true = y_test

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',
label='GS, AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
pickle.dump(forest, open(os.path.join(utils.folder_gen, 'forest.pickle'), 'wb'))

In [ ]:
forest2 = pickle.load(open(os.path.join(utils.folder_gen, 'forest.pickle'), 'rb'))

In [ ]:
len(features[0])

### Try keras ML

In [ ]:
from keras.losses import mean_squared_error
from keras.models import Sequential, Model
from keras.layers import LSTM,Activation,Dense, Lambda, Input, Concatenate, Add, Reshape, Dropout
from keras.optimizers import Adam
from keras.metrics import mean_squared_error
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.engine.topology import Layer
from keras.initializers import RandomNormal
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import multiply
from keras import backend as K
from keras.constraints import non_neg
import tensorflow as tf
import keras

In [ ]:
#Формирование модели
model_number = 0
best_weights_filepath = './best_weights.hdf5'
log_path = './logs_nn/'

In [ ]:
bin=42
def get_model(x_train,
              y_train,
              model_number=0,
              best_weights_filepath=best_weights_filepath,
              log_path='./logs_nn/'):
    '''hyperparams'''
#     learning_rate = 1e-3
#     reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001)
#     optimiser = Adam(lr=learning_rate)
    dense_activation = 'relu'
    dense_1layer_nn_count_coef = 1
    dense_2layer_nn_count_coef = 1
    
    out_len = 1
    feature_amount = len(x_train[0])
    others_count = feature_amount - bin*2
    in_shape = (feature_amount, )
    
    ''' Input '''
    inputs = Input(shape=in_shape, name="in_data")
    inputs_transformed = BatchNormalization()(inputs)
    
    ''' Slices '''
    input_bins = [None]*bin
    for i in range(bin):
        slice_0 = Lambda(lambda x: x[:,i:i+1],
            output_shape=lambda in_shape: (in_shape[0], 1))(inputs_transformed)
        slice_1 = Lambda(lambda x: x[:,bin+i:bin+i+1],
            output_shape=lambda in_shape: (in_shape[0], 1))(inputs_transformed)
        input_bins[i] = keras.layers.concatenate([slice_0, slice_1])

    slice_others = Lambda(lambda x: x[:,-others_count:],
            output_shape=lambda in_shape: (in_shape[0], feature_amount - bin*2))(inputs_transformed)

    ''' Dense for input_bins '''
    pair_denses = [None]*bin
    for i in range(bin):
        input_bins[i]
        pair_denses[i] = Dense(units = 1, activation = dense_activation)(input_bins[i])
    
    ''' Dense others '''
    dense_others = Dense(units = others_count, activation = dense_activation)(slice_others)
    
    ''' Combine Denses '''
    combine_pairs_and_others = keras.layers.concatenate(pair_denses + [dense_others])
    dense_nn_count = bin + others_count
    dropout = Dropout(0.2)(combine_pairs_and_others)
    dense_model = Dense(units = dense_nn_count, activation = dense_activation)(dropout)    
    
    ''' Final Denses '''
    dropout = Dropout(0.1)(dense_model)
    dense_nn_count = bin // 2 + others_count // 2
    dense_model = Dense(units = dense_nn_count, activation = dense_activation)(dense_model)
    output = Dense(units = out_len, activation = 'sigmoid', name='out_states')(dense_model)   
    
    model = Model(inputs, output)
    
    tb_callback = TensorBoard(log_dir=log_path + '/', histogram_freq=0, write_graph=True,
                              write_images=True)
    bm_callback = ModelCheckpoint(best_weights_filepath, monitor='acc', verbose=1, save_best_only=True,
                                  mode='auto')
    
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop', #'sgd'
                  metrics=['accuracy'])
    return [model, [bm_callback, tb_callback]]

In [ ]:
!rm -rf ./logs_nn/

In [ ]:
model_number += 1
[model, callbacks] = get_model(x_train, y_train, log_path = log_path+str(model_number))

In [ ]:
batch_size = 512
epochs = 1800

model.fit(x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
#       validation_data=(x_test, y_test),
      initial_epoch = 724,
      callbacks = callbacks,
      verbose = 1
     )

### save model

In [ ]:
reload(utils)
model.load_weights(best_weights_filepath)
utils.save_model(model, 'matches_f85_th81')

In [ ]:
reload(utils)
[model, [bm_callback, tb_callback]] = utils.load_model('matches_f85_th81', feature_amount = 85)

### Check model trained

In [ ]:
x_watch = x_test
y_watch = y_test
for i in range(40):
    x = np.array(x_watch[i]).reshape((1, -1))
    res = model.predict(x)
    if y_watch[i] == 1:
        print(res, y_watch[i])

In [ ]:
true_mapping = [tuple(x) for x in utils.read_true_mapping().values]
test_lid_rid = list(set(true_mapping) - set(matches))

def get_degs(g, uid):
    degs = []
    for v in g[uid]:
        degs.append(g.degree(v))
    return degs

def double_deg_degs(g, uid):
    degs = []
    for v in g[uid]:
        degs += get_degs(g,v)
    return degs

In [ ]:
import itertools  as it
from fuzzywuzzy import fuzz
import numpy as np
from random import random
reload(utils)

def f1_score(pred, ratio):
    return 2* (pred * ratio) / (pred + ratio)

def res_pred(pred, ratio):
    return pred + ratio

def neigbor_deg_dist(lg, rg, lid_rid, model, forest):
    for lid, rid in lid_rid[140:152]:
        try:
            degs_l = get_degs(lg, lid)
            degs_l2 = double_deg_degs(lg, lid)
            lfname = lg.node[lid]['fname']
            ldegree = lg.degree(lid)
            for vr in [rid] + list(rg[rid].keys())[:4]:
                rfname = rg.node[vr]['fname']
                ratio = fuzz.token_set_ratio(lfname, rfname) / 100

                x = np.array(utils.feature(lg, lid, rg, vr)).reshape((1, -1))
                f, axarr = plt.subplots(nrows = 1, ncols=4, figsize=(14,4))
                axarr[0].hist(degs_l, bins=30)
                axarr[0].set_title('%s \n %s \n degree %d | %d' % (lfname, rg.node[vr]['fname'], ldegree, rg.degree(vr)))

                degs_r = get_degs(rg, vr)
                axarr[1].hist(degs_r, bins=30)
                pred = model.predict(x)
                pred_forest = forest.predict_proba(x)[0][1]
                axarr[1].set_title('model %f \nfores %f \n ratio %d' % (pred, pred_forest, ratio))

                axarr[2].hist(degs_l2, bins=30)
                axarr[2].set_title('avg model ratio: %f' % ((pred + ratio)))

                axarr[3].hist(double_deg_degs(rg, vr), bins=30)

                plt.show()
        except KeyError:
            pass
        print('###########################################################3')
    
neigbor_deg_dist(G1, G2, test_lid_rid,model, forest)